In [4]:
import pyreadstat
import os 
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#lines below let allow multiple results from a line of code to be shown e.g. df.head() + df.columns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
os.chdir('C:\\Users\\Sam Cannon\\Desktop\\R\\Data Sets\\new_datasets_7e\\new_datasets_7e')

In [6]:
#SPSS reading module, need to replace column names though
import savReaderWriter as s
df= pd.DataFrame(list(s.SavReader('C:\\Users\\Sam Cannon\\Desktop\\R\\Data Sets\\new_datasets_7e\\new_datasets_7e\\Lesson 25 Data File 1.sav')))

In [8]:
#rename columns
df = df.set_axis(['Group', 'Difference'], axis=1, inplace=False)

In [14]:
df

,Group,Difference
0,1.0,12.0
1,1.0,-2.0
2,1.0,9.0
3,1.0,3.0
4,1.0,3.0
5,1.0,0.0
6,1.0,3.0
7,1.0,2.0
8,1.0,4.0
9,1.0,1.0


__ANOVA with statsmodels__

- Code was adapted from: https://www.marsja.se/four-ways-to-conduct-one-way-anovas-using-python/ and https://pythonfordatascience.org/anova-python/#test 
- explanations were taken directly from these websites as well, the data is the only difference here
- Using statsmodels, we get a bit more information and enter the model as a regression formula. The general input using this method looks like this:
    - model_name = ols('outcome_variable ~ group1 + group2 + groupN', data=your_data).fit()

- If you dummy code the groups, you have to not include 1 of the groups in the formula. This group’s data will still get captured in the model’s intercept and is the base (control) group. If you use the following method of entering the formula Python takes care of this for you.

In [21]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

#have to use C() with the control group
mod = ols('Difference ~ C(Group)', data=df).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

          sum_sq    df         F   PR(>F)
C(Group)   205.4   2.0  4.835891  0.01603
Residual   573.4  27.0       NaN      NaN


Let’s break down this ANOVA table. The dose row is the between groups effect which is the overall experimental effect. The sum of squares for the model (SSM; value 20.133 in the table) is how much variance is explained by our model. The current model explains a significant amount of variance, F(2,12)= 5.12, p < 0.05. The residual row is the unsystematic variation in the data (SSR; also called the unexplained variance; value 23.600 in the table). In this case, the unsystematic variation represents the natural individual differences in libido and natural different reactions to the drug, Difficile.

In [22]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Difference   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     4.836
Date:                Mon, 22 Jul 2019   Prob (F-statistic):             0.0160
Time:                        14:20:45   Log-Likelihood:                -86.824
No. Observations:                  30   AIC:                             179.6
Df Residuals:                      27   BIC:                             183.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.5000      1.457      2.402      0.023       0.510       6.490
C(Group)[T.2.0]    -5.6000      2.061     -2.717      0.011      -9.829      -1.371
C(Group)[T.3.0]    -5.5000      2.061     -2.669      0.013      -9.729      -1.271
==============================================================================
Omnibus:                        3.156   Durbin-Watson:                   2.135
Prob(Omnibus):                  0.206   Jarque-Bera (JB):                2.729
Skew:                           0.652   Prob(JB):                        0.256
Kurtosis:                       2.304   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(aov_table)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(Group),205.4,2.0,102.700000,4.835891,0.01603,0.263739,0.203648
Residual,573.4,27.0,21.237037,NaN,NaN,NaN,NaN


CALCULATING MODEL EFFECT SIZE
Something that is useful is the effect size. The effect size tells us how much of an impact the experiment will have in the real world. There are a few different effect sizes one can use: eta squared (?2), and omega squared (?2). Omega squared is considered a better measure of effect size than eta squared because it is unbiased in it’s calculation.

Something to note, for some reason R2 is called eta squared within the ANOVA framework. They are the same thing. R2 is a measure of how much variance is explained by the model and is calculated by taking the explained variance (SSM) and dividing it by the total variance (SST; also called total sum of squares). With the total variance (SST) equaling the sum of squares for the model (SSM) plus the sum of square for the residual (SSR). Thus making the equation for R2 and eta squared:

R2 and eta squared = SSM/SST
R2 and eta squared = 20.133/43.733 = 0.460

That means the current model accounts for 46.0% of the variance in contributing to libido. Like just mentioned, within the ANOVA framework, R2 is also called eta squared, and can be interpreted as the amount of explained variance, as well as an effect size measure.

Another thing we need to calculate is the mean squares. The mean squares is desired because it eliminates the bias present in the SSM and SSR, and it is also used to calculate the F-statistic and omega squared. SSM and SSR are biased because they are influenced by the number of values summed to calculated them. To calculate the mean squares, one divides the sum of squares (SSM and SSR) by the degrees of freedom respectively.

MSM= SSM/dfM = 20.135/2 = 10.067
MSR= SSR/dfR = 23.60/12 = 1.967

MSM is the average amount of variance explained by the current model, MSR is the average amount of variance unexplained by the current model. The ratio of MSM to MSR is used to calculate the F-statistic. We don’t need to do this since we already have it, but it’s nice to understand where the numbers come from!

MSM/MSR = 10.067/1.967 = 5.118

The following function calculates the effect sizes mentioned, as well as the mean squares and updates the table!

In [26]:
mod.diagn

{'jb': 2.728816783985189,
 'jbpv': 0.25553180594136615,
 'skew': 0.6516900447777306,
 'kurtosis': 2.3041165547747604,
 'omni': 3.15619079399286,
 'omnipv': 0.20636777282150953,
 'condno': 3.7320508075688776,
 'mineigval': 2.679491924311227}

These are the same diagnostics from the bottom of the regression table from before. The Durban-Watson tests is to detect the presence of autocorrelation (not provided when calling diagnostics this way), Jarque-Bera (jb; jbpv is p-value) tests the assumption of normality, Omnibus (omni; omnipv is p-value) tests the assumption of homogeneity of variance, and the Condition Number (condno) assess multicollinearity. Condition Number values over 20 are indicative of multicollinearity.

If the omnibus test were to be significant, an option on how to handle it would be to use a heteroscedasticity corrected coefficient covariance matrix in the .anova_lm() method. This corrects the calculations to account for the heteroscedasticity present. More information on the method can be found on it’s official documentation page.

__Levene Test__
- check for equal variances, p>.05 means we are good and variances are equal

In [32]:
import scipy.stats as stats
stats.levene(df['Difference'][df['Group'] == 1],
             df['Difference'][df['Group'] == 2],
             df['Difference'][df['Group'] == 3])

LeveneResult(statistic=0.5360670457946722, pvalue=0.5911429509394336)

__Shapiro Test for Normality__
- p > .05 means residuals are normally distributed, t-stat on the left, p-value on the right, our p is less than .05, so the resioduals are not normally distributed

In [33]:
stats.shapiro(mod.resid)

(0.9026545286178589, 0.009759031236171722)

__Post-hoc Testing__
The overall model was significant, now to test which groups differ. Deciding which groups to compare should be theory driven. There are a few different techniques that can be used. Each of these techniques have different ways of controlling for familywise error rate. 3 common methods are:

Fisher’s Least Significant Difference (LSD): Take the groups you want to compare and conduct multiple t-tests. This method requires that the ANOVA model be significant. This method is easy, but receives push back since it doesn’t account for familywise error rate. The argument is that since the overall model was significant, one is protected from increasing the familywise error rate.
Bonferroni correction: Take the alpha the ANOVA was tested at, 0.05, then divide it by the number of planned comparisons. In this case, 0.05/3 = 0.0167. A post-hoc test would have to have an alpha level < 0.0167 to be considered significant. To test the groups, conduct multiple t-tests, but set the alpha value to the corrected value. This method is quick, but often considered too conservative.
Tukey’s HSD: Method also controls for familywise error rate with a different method than Bonferroni, and is also considered conservative.
There are many other techniques out there that can be used for post-hoc testing each with different guidelines for when they should be used, you are encouraged to learn about them!

__TUKEY’S HSD POST-HOC COMPARISON__

In [34]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df['Difference'], df['Group'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower    upper  reject
----------------------------------------------
 1.0    2.0     -5.6   -10.7089 -0.4911  True 
 1.0    3.0     -5.5   -10.6089 -0.3911  True 
 2.0    3.0     0.1    -5.0089   5.2089 False 
----------------------------------------------


The Tukey HSD post-hoc comparison test controls for type I error and maintains the familywise error rate at 0.05 (FWER= 0.05 top of the table). The group1 and group2 columns are the groups being compared, the meandiff column is the difference in means of the two groups being calculated as group2 – group1, the lower/upper columns are the lower/upper boundaries of the 95% confidence interval, and the reject column states whether or not the null hypothesis should be rejected. Unfortunately, this method currently does not provide the t-statistic so treatment effect size cannot be calculated.

__BONFERRONI CORRECTION POST-HOC COMPARISON__
First the corrected p-value needs to be calculated. This can be done using the formula:

p-value/# of comparisons = 0.05/3 = 0.01667

Now the t-tests that are conducted have to have a p-value less than 0.01667 in order to be considered significant.

In [36]:
stats.ttest_ind(df['Difference'][df['Group'] == 1], df['Difference'][df['Group'] == 2])


stats.ttest_ind(df['Difference'][df['Group'] == 1], df['Difference'][df['Group'] == 3])


stats.ttest_ind(df['Difference'][df['Group'] == 2], df['Difference'][df['Group'] == 3])

Ttest_indResult(statistic=3.050011616952708, pvalue=0.0068922723946386885)

Ttest_indResult(statistic=2.532474592535423, pvalue=0.020849157649945604)

Ttest_indResult(statistic=-0.046351743495682274, pvalue=0.9635402876217463)